In [1]:
import os
import sys
import mlflow
import time
import gc
import contextlib

import tensorflow
import pandas as pd
import seaborn as sns
import numpy as np
import keras.backend as K

from io import StringIO
from hyperopt import fmin, tpe, hp, Trials, space_eval
from matplotlib import pyplot as plt
from dotenv import load_dotenv
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, make_scorer

sys.path.append("../")

from scripts.mlflow_functions import train_and_log, train_and_log_keras
from models.scorer import home_credit_scoring_fn, home_credit_scorer
from models.scorer import home_credit_loss_fn_keras


load_dotenv()
sns.color_palette('colorblind')
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100

client = mlflow.MlflowClient(tracking_uri=os.path.abspath("../mlruns/"))

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))


In [2]:
try:
    mlflow.create_experiment(name="home_credit_model")
except mlflow.MlflowException:
    mlflow.set_experiment(experiment_name="home_credit_model")


In [3]:
df_model = pd.read_pickle(filepath_or_buffer="../data/df_hc_nm.pkl")


In [4]:
df_model.head()


,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,PREV_PRODUCT_COMBINATION_Cash X-Sell: low_mean,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_mean,PREV_PRODUCT_COMBINATION_POS household with interest_mean,PREV_PRODUCT_COMBINATION_POS household without interest_mean,PREV_PRODUCT_COMBINATION_POS industry with interest_mean,PREV_PRODUCT_COMBINATION_POS industry without interest_mean,PREV_PRODUCT_COMBINATION_POS mobile with interest_mean,PREV_PRODUCT_COMBINATION_POS mobile without interest_mean,PREV_PRODUCT_COMBINATION_POS other with interest_mean,PREV_number_applications
0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0
26,0,0,0,0,112500.0,979992.0,27076.5,702000.0,0.018029,-18724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,0,0,0,0,202500.0,1193580.0,35028.0,855000.0,0.025164,-17482,...,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,6.0
42,0,0,1,0,135000.0,288873.0,16258.5,238500.0,0.007305,-13384,...,0.0,0.2,0.2,0.0,0.0,0.0,0.400000,0.0,0.0,5.0
45,1,0,0,1,90000.0,180000.0,9000.0,180000.0,0.009334,-7974,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,2.0


In [5]:
df_model.rename(columns={"TARGET": "Payment_difficulties"}, inplace=True)

target_col = "Payment_difficulties"


# Modelisation and experimentation

- Threshold optimization : The first xgboost run will be used to adjust the threshold of the scorer
- Models : Ensemble methods, handle missing values and don't require scaling
    - xGBoost
    - CatBoost
    - LightGBM
- Optimization : Best method will be determined via benchmark
    - Hyperopt
- Logging via MlFlow :
    - Metrics
    - Custom metric tuning
    - AUROC, confusion matrix
    - Global feature importance
    - Model
    - Method of undersampling
    - Version (handling of nans.)

<hr>


In [6]:
# Splitting the data into training, validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    df_model.drop(columns=target_col),
    df_model[target_col],
    test_size=0.3,
    random_state=123
    )

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size=0.25,
    random_state=123
    )


# 0 : Overfitting prevention
- Overfitting on tree based classifiers is often a problem. The weights of the weak estimators might introduce some bias in the final results
- There are several steps we can take to prevent overfitting :
    - A hold out set for validation : Splitting the training data (cell above) in a validation set makes testing the model on **unseen** data, this makes evaluation of the model more neutral
    - Regularization parameters : among the hyperparameters tested, some parameters are set specifically to avoid overfitting : `learning_rate`, L1 & L2 regularization for example, are aggressively set higher than default.
    - CrossValidation : hyperopt supports crossvalidation, as does GridSearchCV (but hyperopt is way faster in this scenario). Hyperparameter tuning is crossvalidated with `cross_val_score` set to `cv=5`
    - DNN specific : two layers of dropout (30% then 20%) disables random neurons to avoid overfitting
    - DNN specific : the model features the validation scores and metrics (in addition to the train/test metrics). This tests the ability of the classifier on unseen data and allows us to see when overfitting occurs. We can thus select the best model based on the validation score and not the training score.

# 1 xGBoost

## This will aslo serve as calibration for the threshold of the metric
- We will use the optimized xGboost to predict the probabilities of classification and apply the threshold finder onto it
- This might mean that xGboost will be reran twice 

### Hyperopt optimization


In [23]:
adjusted_threshold = 0.6004038524356866  # RUN 3
# Confirmed final threshold


In [24]:
xgb_param_space = {
    "n_estimators": hp.choice("n_estimators", np.arange(200, 800, 25, dtype=int)),
    "max_depth": hp.choice("max_depth", np.arange(3, 9, dtype=int)),
    "learning_rate": hp.loguniform("learning_rate", -7, -3),
    "min_child_weight": hp.quniform("min_child_weight", 1, 6, 0.1),
    "reg_alpha": hp.loguniform("reg_alpha", -6, -3),
    "reg_lambda": hp.loguniform("reg_lambda", -6, -3),
    "subsample": hp.uniform("subsample", 0.5, 0.9),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.3, 0.9),
    "nthread": -1
}


def objective(params):
    model = XGBClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        learning_rate=params["learning_rate"],
        min_child_weight=params["min_child_weight"],
        reg_alpha=params["reg_alpha"],
        reg_lambda=params["reg_lambda"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        nthread=params["nthread"]
    )
    scores = cross_val_score(
        model,
        X_train,
        y_train,
        cv=5,
        scoring=make_scorer(home_credit_scoring_fn, threshold=adjusted_threshold)
        )

    loss = -np.mean(scores)
    return {"loss": loss, "status": "ok"}


trials = Trials()

time_start_hopt = time.perf_counter()
best = fmin(objective, space=xgb_param_space, algo=tpe.suggest, max_evals=20, trials=trials)
time_end_hopt = time.perf_counter()

print(f"Hyperopt search time: {time_end_hopt - time_start_hopt} seconds.")


100%|██████████| 20/20 [52:45<00:00, 158.27s/trial, best loss: -4.8676429400265295]
Hyperopt search time: 3165.2632431249367 seconds.


In [25]:
best_params_xgb = space_eval(xgb_param_space, best)
gc.collect()
print(best_params_xgb)


{'colsample_bytree': 0.5664064462789224, 'learning_rate': 0.001980282806150775, 'max_depth': 4, 'min_child_weight': 5.300000000000001, 'n_estimators': 475, 'nthread': -1, 'reg_alpha': 0.045887098756441945, 'reg_lambda': 0.004368002328795769, 'subsample': 0.6719121773597623}


In [26]:
mlflow.set_experiment(experiment_name="home_credit_model")

best_params_xgb_log = {
    "colsample_bytree": 0.5664064462789224,
    "learning_rate": 0.001980282806150775,
    "max_depth": 4,
    "min_child_weight": 5.300000000000001,
    "n_estimators": 475,
    "reg_alpha": 0.045887098756441945,
    "reg_lambda": 0.004368002328795769,
    "subsample": 0.6719121773597623
    }

xgb_metrics, xgb_clf = train_and_log(
    estimator=XGBClassifier,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    score_threshold=adjusted_threshold,
    dataset_version="nans_kept_shap_reduced",
    imb_method="near_miss_one",
    na_thresh=0,
    params=best_params_xgb_log,
    model_name="xbgoost_hyperopt_best_cv",
)


ntree_limit is deprecated, use `iteration_range` or model slicing instead.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
eval_metric is not saved in Scikit-Learn meta.
Successfully registered model 'xbgoost_hyperopt_best_cv'.
2023/03/07 14:18:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: xbgoost_hyperopt_best_cv, version 1
Created version '1' of model 'xbgoost_hyperopt_best_cv'.
Registered model 'xbgoost_hyperopt_best_cv' already exists. Creating a new version of this model...
2023/03/07 14:18:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: xbgoost_hyperopt_best_cv, version 2
Created version '2' of model 'xbgoost_hyperopt_best_cv'.


In [22]:
# Treshold adjusted, skipping

# # Adjusting threshold - then redoing the search

# y_proba = xgb_clf.predict_proba(X_val)

# threshold_space = hp.uniform("threshold", 0, 1)


# def objective(threshold):
#     # Convert predicted probabilities to binary labels using the threshold
#     y_pred_binary = (y_proba[:, 1] >= threshold).astype(int)

#     # Calculate the score using the custom scoring function
#     score = home_credit_scoring_fn(y_true=y_val, y_pred=y_pred_binary)

#     return {"loss": -score, "status": "ok"}


# # Run the hyperparameter optimization
# best = fmin(objective, threshold_space, algo=tpe.suggest, max_evals=100)

# # Get the best threshold and score
# best_threshold = best["threshold"]
# best_score = -objective(best_threshold)["loss"]

# print("Best threshold:", best_threshold)
# print("Best score:", best_score)


100%|██████████| 100/100 [00:00<00:00, 1010.60trial/s, best loss: -9.999999999773346]
Best threshold: 0.6004038524356866
Best score: 9.999999999773346


In [27]:
y_val_proba = xgb_clf.predict_proba(X_val)
y_val_pred = xgb_clf.predict(X_val)

report = classification_report(y_val, y_val_pred)
val_score = home_credit_scoring_fn(y_val, y_pred=y_val_pred)

print(f"Validation score: {val_score}")
print(report)


Validation score: 4.89476167980676
              precision    recall  f1-score   support

           0       0.77      0.72      0.74      4270
           1       0.74      0.79      0.77      4412

    accuracy                           0.76      8682
   macro avg       0.76      0.76      0.76      8682
weighted avg       0.76      0.76      0.76      8682



### Observations :

- The report is based on a validation/hold-out set, this is data the classifier was not trained/tested on. This shows the ability of the model on unseen data.
- The metrics are overall pretty good. As we are interested mainly on false positives, recall and f1 are the two metrics we want to keep an eye on. Both are above 0.7.
- This also shows the influence of the custom metric on the results : the recall and F1 are both higher when classifying data labelled as 1 (i. e. : the client is a risk). The training on the custom metric introduces some bias, as requested. This classifier is particularly well trained to find client who would present a risk, at the cost of misindentifying some that do not present a high risk.

# 2 : Catboost :

In [28]:
catboost_param_space = {
    "iterations": hp.choice("iterations", np.arange(200, 800, 25, dtype=int)),
    "depth": hp.choice("depth", np.arange(3, 9, dtype=int)),
    "learning_rate": hp.loguniform("learning_rate", -7, -3),
    "l2_leaf_reg": hp.loguniform("l2_leaf_reg", -6, -3),
    "subsample": hp.uniform("subsample", 0.5, 0.9),
    "colsample_bylevel": hp.uniform("colsample_bylevel", 0.3, 0.9),
    "verbose": False
    }


def objective_cb(params):
    model = CatBoostClassifier(
        iterations=params["iterations"],
        depth=params["depth"],
        subsample=params["subsample"],
        colsample_bylevel=params["colsample_bylevel"],
        learning_rate=params["learning_rate"],
        l2_leaf_reg=params["l2_leaf_reg"],
        verbose=params["verbose"],
        thread_count=-1,
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    scores = cross_val_score(
        model,
        X_train,
        y_train,
        cv=5,
        scoring=make_scorer(home_credit_scoring_fn, threshold=adjusted_threshold)
        )
    loss = -np.mean(scores)
    return {"loss": loss, "status": "ok"}


trials = Trials()

time_start_hopt = time.perf_counter()
best_cb = fmin(objective_cb, space=catboost_param_space, algo=tpe.suggest, max_evals=20, trials=trials)
time_end_hopt = time.perf_counter()

print(f"Hyperopt search time: {time_end_hopt - time_start_hopt} seconds.")


100%|██████████| 20/20 [14:19<00:00, 42.95s/trial, best loss: -5.33357793702625] 
Hyperopt search time: 859.0927498750389 seconds.


In [29]:
best_params_cb = space_eval(catboost_param_space, best_cb)
print(best_params_cb)


{'colsample_bylevel': 0.6085039314472103, 'depth': 3, 'iterations': 250, 'l2_leaf_reg': 0.0028892223893425576, 'learning_rate': 0.0010208031321721773, 'subsample': 0.7210157471199539, 'verbose': False}


In [30]:
mlflow.set_experiment(experiment_name="home_credit_model")

best_params_cb_log = {
    "colsample_bylevel": 0.6085039314472103,
    "depth": 3,
    "iterations": 250,
    "l2_leaf_reg": 0.0028892223893425576,
    "learning_rate": 0.0010208031321721773,
    "subsample": 0.7210157471199539,
    "verbose": False
    }

cb_metrics, cb_clf = train_and_log(
    estimator=CatBoostClassifier,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    score_threshold=adjusted_threshold,
    dataset_version="nans_kept_shap_reduced",
    imb_method="near_miss_one",
    na_thresh=0,
    params=best_params_cb_log,
    model_name="cb_hyperopt_best_cv",
)


No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Successfully registered model 'cb_hyperopt_best_cv'.
2023/03/07 14:35:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: cb_hyperopt_best_cv, version 1
Created version '1' of model 'cb_hyperopt_best_cv'.
Registered model 'cb_hyperopt_best_cv' already exists. Creating a new version of this model...
2023/03/07 14:35:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: cb_hyperopt_best_cv, version 2
Created version '2' of model 'cb_hyperopt_best_cv'.


In [31]:
# classification report

y_val_proba_cb = cb_clf.predict_proba(X_val)
y_val_pred_cb = cb_clf.predict(X_val)

report_cb = classification_report(y_val, y_val_pred_cb)
val_score_cb = home_credit_scoring_fn(y_val, y_val_pred_cb)

print(f"Validation score: {val_score_cb}")
print(report_cb)


Validation score: 5.213147410150871
              precision    recall  f1-score   support

           0       0.71      0.69      0.70      4270
           1       0.71      0.73      0.72      4412

    accuracy                           0.71      8682
   macro avg       0.71      0.71      0.71      8682
weighted avg       0.71      0.71      0.71      8682



### Obsevation :
- Catboost seems to be less performant than xGboost, although still quite efficient. It exhibits the same behaviour (better at identifying false positives)
- The model *might* perform better if the preprocessing wasnt featuring OneHotEncoding as CatBoost is specifically powerful on categorical data. However, the models should be tested on the same datasets for comparison purposes.

# 3 : Dense Neural Network :
- DNNs perform better on scaled data (while ensemble methods are not impacted by scaling). The dataset used to train the DNN is the same as the one used by tree based classifiers, but with nans imputed and scaled features.

In [32]:
df_model_scaled = pd.read_pickle(filepath_or_buffer="../data/df_train_hc_nm_imputed_scaled.pkl")
df_model_scaled.rename(columns={"TARGET": "Payment_difficulties"}, inplace=True)


## Redoing the split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    df_model_scaled.drop(columns=["Payment_difficulties"]),
    df_model_scaled["Payment_difficulties"],
    test_size=0.3,
    random_state=123,
    stratify=df_model_scaled["Payment_difficulties"]
    )

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.25,
    random_state=123,
    stratify=y_train
    )


## DNN : 

- input of size feature len
- 512 dense
- dropout 30%
- 256 dense
- dropout 20%
- Sigmoid for binary clf
- Validation set to track overfitting


In [34]:
# Adjusting loss function with threshold : 

def home_credit_loss_fn_keras(y_true, y_pred, threshold=adjusted_threshold):
    """
    Custom loss function meant to modify binary crossentropy to use the costs and threshold of
    the home_credit metrics, specific to keras
    """
    fn_cost = 10
    fp_cost = 1

    y_pred_binary = K.cast(K.greater_equal(y_pred, threshold), "float32")

    fp = K.sum(K.cast(K.equal(y_pred_binary, 1) & K.equal(y_true, 0), "float32"))
    fn = K.sum(K.cast(K.equal(y_pred_binary, 0) & K.equal(y_true, 1), "float32"))

    loss = K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    loss = loss + (fn * fn_cost + fp * fp_cost) / (fn + fp + 1e-7)  # Prevents zero division

    return loss


In [35]:
recall = tensorflow.keras.metrics.Recall()

# Input layer
inputs = tensorflow.keras.Input(shape=(X_train.shape[1],))

# Hidden layers
dense_1 = tensorflow.keras.layers.Dense(512, activation="relu")(inputs)
dropout_30 = tensorflow.keras.layers.Dropout(0.3)(dense_1)
dense_2 = tensorflow.keras.layers.Dense(256, activation="relu")(dropout_30)
dropout_20 = tensorflow.keras.layers.Dropout(0.2)(dense_2)

# Output layer
outputs = tensorflow.keras.layers.Dense(1, activation="sigmoid")(dropout_20)

model = tensorflow.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="adam", loss=home_credit_loss_fn_keras, metrics=[recall])


In [36]:
model_summary = model.summary()

summary_buffer = StringIO()

with contextlib.redirect_stdout(summary_buffer):
    model.summary()

model_summary = summary_buffer.getvalue()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 283)]             0         
                                                                 
 dense (Dense)               (None, 512)               145408    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 276,993
Trainable params: 276,993
Non-trainable

In [37]:
start = time.perf_counter()

history = model.fit(
    X_train,
    y_train,
    epochs=35,
    batch_size=64,
    validation_data=(X_val, y_val)
    )

end = time.perf_counter()

training_time = end - start


Epoch 1/35


2023-03-07 14:35:53.027736: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


407/407 [==============================] - 2s 3ms/step - loss: 7.8201 - recall: 0.7030 - val_loss: 9.1378 - val_recall: 0.6676
Epoch 2/35
407/407 [==============================] - 1s 3ms/step - loss: 7.7566 - recall: 0.7487 - val_loss: 7.8400 - val_recall: 0.7431
Epoch 3/35
407/407 [==============================] - 1s 3ms/step - loss: 7.6662 - recall: 0.7629 - val_loss: 7.7491 - val_recall: 0.7554
Epoch 4/35
407/407 [==============================] - 1s 3ms/step - loss: 7.7017 - recall: 0.7687 - val_loss: 7.0881 - val_recall: 0.7982
Epoch 5/35
407/407 [==============================] - 1s 3ms/step - loss: 7.5705 - recall: 0.7808 - val_loss: 7.0113 - val_recall: 0.7954
Epoch 6/35
407/407 [==============================] - 1s 3ms/step - loss: 7.5005 - recall: 0.7960 - val_loss: 7.0905 - val_recall: 0.7717
Epoch 7/35
407/407 [==============================] - 1s 3ms/step - loss: 7.5011 - recall: 0.8049 - val_loss: 7.2546 - val_recall: 0.7643
Epoch 8/35
407/407 [=========================

In [38]:
def get_best_model(history):
    """
    Returns the best Keras model based on validation loss from the training history.
    """
    val_loss = history.history["val_loss"]
    best_epoch = val_loss.index(min(val_loss))
    best_model = tensorflow.keras.models.clone_model(model)
    best_model.build(input_shape=X_train.shape[1:])
    best_model.compile(
        optimizer='adam',
        loss=home_credit_loss_fn_keras,
        metrics=[tensorflow.keras.metrics.Recall()]
        )
    best_model.set_weights(history.model.get_weights())
    return best_model, best_epoch



In [39]:
best_model, best_epoch = get_best_model(history=history)


In [40]:
best_score = best_model.evaluate(x=X_val, y=y_val)[0]
print(f"Best model achieved after : {best_epoch} epochs")


272/272 [==============================] - 0s 647us/step - loss: 7.0477 - recall_1: 0.7196
Best model achieved after : 22 epochs


In [42]:
mlflow.set_experiment(experiment_name="home_credit_model")

metrics, model = train_and_log_keras(
    model=best_model,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    training_time=training_time,
    model_summary=model_summary,
    history=history,
    model_name="DNN",
    dataset_version="Nans_imputed_scaled",
    imb_method="nm1",
    home_credit_score=best_score,
    feature_list=X_train.columns.tolist()
)


466/466 [==============================] - 0s 619us/step


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
2023/03/07 14:44:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/3s/s8sp6jwn6qs02jfxbgjc7c_40000gn/T/tmphy3sq8a5/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/3s/s8sp6jwn6qs02jfxbgjc7c_40000gn/T/tmphy3sq8a5/model/data/model/assets
2023/03/07 14:44:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/3s/s8sp6jwn6qs02jfxbgjc7c_40000gn/T/tmphy3sq8a5/model, flavor: tensorflow), fall back to return ['tensorflow==2.11.0']. Set logging level to DEBUG to see the full traceback.
Successfully registered model 'DNN_TEST'.
2023/03/07 14:44:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: DNN_TEST, version 1
Created version '1' of model 'DNN_TEST'.
Registered model 'DNN_TEST' already exists. Creating a new version of this model...
2023/03/07 14:44:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: DNN_TEST, version 2
Created version '2' of model 'DNN